<a href="https://colab.research.google.com/github/vincebarokie/sales-prediction/blob/main/sales_prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Project : Sales Prediction

This project will be a sales prediction for food items sold at various stores. The goal of this is to help the retailer understand the properties of products and outlets that play crucial roles in increasing sales.

## Data Loading and Cleanup

* Load the CSV file to a DataFrame
* Explore and do necessary data preparation and cleanup

### Mounting and Loading

* Mount the drive
* Import libraries
* Load CSV as DataFrame

In [97]:
# mount drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [98]:
# import libraries
import pandas as pd
import numpy as np
# import datawig

In [99]:
# load sales_predictions.csv file to df
filename = '/content/drive/MyDrive/CodingDojo_DS/Project/sales_predictions.csv'
df = pd.read_csv(filename)
df.head()

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales
0,FDA15,9.30,Low Fat,0.016047,Dairy,249.8092,OUT049,1999,Medium,Tier 1,Supermarket Type1,3735.1380
1,DRC01,5.92,Regular,0.019278,Soft Drinks,48.2692,OUT018,2009,Medium,Tier 3,Supermarket Type2,443.4228
2,FDN15,17.50,Low Fat,0.016760,Meat,141.6180,OUT049,1999,Medium,Tier 1,Supermarket Type1,2097.2700
3,FDX07,19.20,Regular,0.000000,Fruits and Vegetables,182.0950,OUT010,1998,NaN,Tier 3,Grocery Store,732.3800
4,NCD19,8.93,Low Fat,0.000000,Household,53.8614,OUT013,1987,High,Tier 3,Supermarket Type1,994.7052


### DataFrame Shape and Info

Get how many rows and columns, the data types and missing data. 

In [100]:
# How many rows and columns? 
df.shape

(8523, 12)

In [101]:
# dtypes of each variable
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8523 entries, 0 to 8522
Data columns (total 12 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Item_Identifier            8523 non-null   object 
 1   Item_Weight                7060 non-null   float64
 2   Item_Fat_Content           8523 non-null   object 
 3   Item_Visibility            8523 non-null   float64
 4   Item_Type                  8523 non-null   object 
 5   Item_MRP                   8523 non-null   float64
 6   Outlet_Identifier          8523 non-null   object 
 7   Outlet_Establishment_Year  8523 non-null   int64  
 8   Outlet_Size                6113 non-null   object 
 9   Outlet_Location_Type       8523 non-null   object 
 10  Outlet_Type                8523 non-null   object 
 11  Item_Outlet_Sales          8523 non-null   float64
dtypes: float64(4), int64(1), object(7)
memory usage: 799.2+ KB


### Add Grouping Columns

There's a patteren used in the Item_Identifier column and could be useful to add new column with just the first 2 letters. 

In [102]:
df['Item_Group'] = df['Item_Identifier'].map(lambda x: x[:2])
df[['Item_Identifier','Item_Group']].head()

,Item_Identifier,Item_Group
0,FDA15,FD
1,DRC01,DR
2,FDN15,FD
3,FDX07,FD
4,NCD19,NC


### Remove Duplicates

* drop all duplicate rows in the DataFrame and keep first existing values if any

In [103]:
# drop duplicates if there are any
cols = df.columns.to_list()
# df = df.drop_duplicates(subset=cols, keep='first')
# df.info()
df.duplicated(subset=cols, keep='first').sum()

0

### Identify And Address Missing Values

As we can see there are missing values in a couple of columns below > **Item_Weight** and **Outlet_Size**

Take note that **Outlet_Size** is a categorical varibale

In [104]:
# find which variables has missing values
df.isnull().sum()

Item_Identifier                 0
Item_Weight                  1463
Item_Fat_Content                0
Item_Visibility                 0
Item_Type                       0
Item_MRP                        0
Outlet_Identifier               0
Outlet_Establishment_Year       0
Outlet_Size                  2410
Outlet_Location_Type            0
Outlet_Type                     0
Item_Outlet_Sales               0
Item_Group                      0
dtype: int64

Item_Weight variable is numeric, therefore replacing the missing values with the it's average weight is a more locical approach. 

In [105]:
# replacing Item_Weight missing values with the mean value
iw_mean = df['Item_Weight'].mean()
df['Item_Weight'] = df['Item_Weight'].fillna(iw_mean)
df.isnull().sum()

Item_Identifier                 0
Item_Weight                     0
Item_Fat_Content                0
Item_Visibility                 0
Item_Type                       0
Item_MRP                        0
Outlet_Identifier               0
Outlet_Establishment_Year       0
Outlet_Size                  2410
Outlet_Location_Type            0
Outlet_Type                     0
Item_Outlet_Sales               0
Item_Group                      0
dtype: int64

Since Outlet_Size is a categorical variable we are using the mode value as the imputed value. First we get the mode into a series, then we merge that to our main DataFrame. Since we did our merge in a column instead of the index, both columns will be reatained with suffix of _x and _y. We will then drop one of them and rename the column we will be keeping to `Outlet_Size`

In [106]:
# get the mode into a series.
identifier_mode = pd.DataFrame(df.groupby('Outlet_Type')['Outlet_Size'].apply(pd.Series.mode)).reset_index().drop('level_1',axis=1)

# merge the series to the main dataframe, drop and rename necessary columns
clean_df = df.merge(identifier_mode, on='Outlet_Type')
clean_df['Outlet_Size_x'].fillna(clean_df['Outlet_Size_y'], inplace=True)
clean_df = clean_df.rename(columns={'Outlet_Size_x': 'Outlet_Size'}).drop('Outlet_Size_y', axis=1)

# confirm that clean_df has no more missing values
clean_df.isna().sum()

Item_Identifier              0
Item_Weight                  0
Item_Fat_Content             0
Item_Visibility              0
Item_Type                    0
Item_MRP                     0
Outlet_Identifier            0
Outlet_Establishment_Year    0
Outlet_Size                  0
Outlet_Location_Type         0
Outlet_Type                  0
Item_Outlet_Sales            0
Item_Group                   0
dtype: int64

### Fix Incosistent categories of Data

* Check for inconsistencies
* Fix inconsistencies

In [107]:
# check inconsistencies in the categorical values
cat_df = clean_df.select_dtypes(include="object")
for col in list(cat_df):
  print(col + ":")
  print(" > Count of unique values:" + str(len(cat_df[col].unique())))
  print(cat_df[col].unique())

Item_Identifier:
 > Count of unique values:1559
['FDA15' 'FDN15' 'NCD19' ... 'FDN52' 'FDK57' 'FDE52']
Item_Fat_Content:
 > Count of unique values:5
['Low Fat' 'Regular' 'low fat' 'reg' 'LF']
Item_Type:
 > Count of unique values:16
['Dairy' 'Meat' 'Household' 'Snack Foods' 'Frozen Foods'
 'Fruits and Vegetables' 'Breakfast' 'Hard Drinks' 'Breads' 'Soft Drinks'
 'Health and Hygiene' 'Canned' 'Baking Goods' 'Starchy Foods' 'Others'
 'Seafood']
Outlet_Identifier:
 > Count of unique values:10
['OUT049' 'OUT013' 'OUT045' 'OUT017' 'OUT046' 'OUT035' 'OUT018' 'OUT010'
 'OUT019' 'OUT027']
Outlet_Size:
 > Count of unique values:3
['Medium' 'High' 'Small']
Outlet_Location_Type:
 > Count of unique values:3
['Tier 1' 'Tier 3' 'Tier 2']
Outlet_Type:
 > Count of unique values:4
['Supermarket Type1' 'Supermarket Type2' 'Grocery Store'
 'Supermarket Type3']
Item_Group:
 > Count of unique values:3
['FD' 'NC' 'DR']


In [108]:
# fix inconsistencies found in Item_Fat_Content column
clean_df['Item_Fat_Content'] = clean_df['Item_Fat_Content'].replace(['low fat', 'LF', 'reg'],['Low Fat','Low Fat','Regular'])
clean_df['Item_Fat_Content'].unique()

array(['Low Fat', 'Regular'], dtype=object)

### Obtain the summarry statistics

* Summary statistics of each numerical column

In [109]:
# summary statistics of each numerical columns
clean_df.describe()

,Item_Weight,Item_Visibility,Item_MRP,Outlet_Establishment_Year,Item_Outlet_Sales
count,8523.000000,8523.000000,8523.000000,8523.000000,8523.000000
mean,12.857645,0.066132,140.992782,1997.831867,2181.288914
std,4.226124,0.051598,62.275067,8.371760,1706.499616
min,4.555000,0.000000,31.290000,1985.000000,33.290000
25%,9.310000,0.026989,93.826500,1987.000000,834.247400
50%,12.857645,0.053931,143.012800,1999.000000,1794.331000
75%,16.000000,0.094585,185.643700,2004.000000,3101.296400
max,21.350000,0.328391,266.888400,2009.000000,13086.964800
